<a href="https://colab.research.google.com/github/adamxuuuu/figma-pie/blob/master/gdrive_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 和Google Apps连接认证

### 每次重新连接都需要运行并获取验证码


In [ ]:
# This block will authenticate you and will 
# Allow you to interact with Google Sheets 
# And other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

# Aquire credentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

## 从Google Sheet上拿到所需的Sheets


---


#### Crawled | Attributes | Backlog

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import matplotlib.pyplot as plt

# Open workbook
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1GQc_QEqIe9rW37UL7QB9YBnZ_-2kjhVukxM0JyRSor8')
# Open the Backlog spread sheet
backlog = wb.worksheet('Backlog')
# Open the Statistics spread sheet
stats = wb.worksheet('Statistics')
# Open the Crawled spread sheet
crawled = wb.worksheet('Crawled')

## 从Figma上面抓取字段信息，并更新Google Sheet


## 获取页面所需的所有爬取字段，并更新Google Sheet


In [ ]:
attr_need = set()
backlog_data = backlog.get_all_values()
# Remove header row
for row in backlog_data[1:]: 
    # Select colums 3 to 5 -> [2:5)
    for item in row[2:5]:
        attr_need.add(item)
attr_need.remove('')

# Select the range
rng = 'I2:I{}'.format(len(attr_need) + 1)
cell_list = stats.range(rng)

# Populate the list
attr_list = sorted(attr_need, reverse=True)
for cell in cell_list:
    cell.value = attr_list.pop()

# Update in batch
stats.update_cells(cell_list)

{'spreadsheetId': '1GQc_QEqIe9rW37UL7QB9YBnZ_-2kjhVukxM0JyRSor8',
 'updatedCells': 11,
 'updatedColumns': 1,
 'updatedRange': 'Statistics!I2:I12',
 'updatedRows': 11}

## 根据所需爬取字段计算页面可行性，并更新Google Sheet


---


#### 51_job |


In [ ]:
# Get all data from the Crawled sheet 
# And extract attributes from the '51_job' column
crwaled_data = crawled.get_all_values()
attr_51 = []
for row in crwaled_data[1:]:
    if row[5]:
        attr_51.append(row[5])

# Get all values from the Backlog sheet
rows = backlog.get_all_values()
res = dict()
for row in rows[1:]:
    # print(row[2:5])
    available = True
    isEmpty = True
    for attr in row[2:5]:
        if attr:
            isEmpty = False
        if attr not in attr_51:
            available = False
            break
    res[row[1]] = available or not isEmpty

# Select the range
rng = 'L2:M{}'.format(len(res) + 1)
cell_list = stats.range(rng)

# Populate the list
idx = 0
for k, v in res.items():
    cell_list[idx].value = k
    cell_list[idx + 1].value = v
    idx += 2


# Update in batch
stats.update_cells(cell_list)

{'spreadsheetId': '1GQc_QEqIe9rW37UL7QB9YBnZ_-2kjhVukxM0JyRSor8',
 'updatedCells': 96,
 'updatedColumns': 2,
 'updatedRange': 'Statistics!L2:M49',
 'updatedRows': 48}